# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-02-17 22:39:38] INFO utils.py:148: Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-02-17 22:39:38] INFO utils.py:151: Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-02-17 22:39:38] INFO utils.py:164: NumExpr defaulting to 16 threads.


<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.


[2026-02-17 22:39:41] INFO server_args.py:1830: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=120 avail_mem=15.57 GB):   5%|▌         | 1/20 [00:00<00:03,  5.65it/s]

Capturing batches (bs=72 avail_mem=15.56 GB):  20%|██        | 4/20 [00:00<00:01, 15.72it/s]

Capturing batches (bs=24 avail_mem=15.55 GB):  60%|██████    | 12/20 [00:00<00:00, 23.96it/s]

Capturing batches (bs=1 avail_mem=15.55 GB):  95%|█████████▌| 19/20 [00:00<00:00, 27.26it/s]

Capturing batches (bs=1 avail_mem=15.55 GB): 100%|██████████| 20/20 [00:00<00:00, 24.00it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Mark. I am a 51-year-old man from Los Angeles, California. I love my job at a music shop. I work there in an office next to the store. I have a good relationship with the people working there, but I don’t know them very well. I do not have a girlfriend or any girlfriend-like relationships. I like to go out to restaurants and cafes with friends. I have two young kids. One is a 5-year-old boy and the other is a 3-year-old girl. I usually take my kids to the park on Saturdays. They love to play there. I go to a nearby
Prompt: The president of the United States is
Generated text:  proposing a series of changes to the American military. According to the proposal, the military would be divided into four branches: the Army, the Navy, the Air Force, and the Marines. Each branch would have a different number of troops. The Army has 40,000 troops, the Navy has 35,000 troops, the Air Force has 25,000 troops, and the Marines have 20,000 troops. If the pre

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [Age] year old [Occupation]. I'm a [Skill] who has been [Number of Years] years in the field of [Field of Interest]. I'm a [Skill] who has been [Number of Years] years in the field of [Field of Interest]. I'm a [Skill] who has been [Number of Years] years in the field of [Field of Interest]. I'm a [Skill] who has been [Number of Years] years in the field of [Field of Interest]. I'm a [Skill] who has been [Number of Years] years in

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. It is the largest city in France and the second-largest city in the European Union. Paris is known for its rich history, beautiful architecture, and vibrant culture. It is home to many famous landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral. Paris is also a major center for finance, fashion, and entertainment. It is a popular tourist destination and a major economic hub in Europe. The city is home to many important institutions such as the French Academy of Sciences, the French National Library, and the French National Museum of Modern Art. Paris is a city of contrasts, with its

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and decision-making processes.

2. Enhanced creativity and innovation: AI is likely to become more capable of generating creative and innovative ideas, as well as performing tasks that require human creativity and ingenuity.

3. Increased ethical and social implications: As AI becomes more integrated into our lives, there will be increased ethical and social implications to consider, such as the potential for AI to perpetuate or exacerbate existing social inequalities.

4. Greater reliance on AI for



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name]. I'm a [X] and I've been in the [Y] industry for [Z] years. I'm passionate about [A], and I'm always ready to learn and grow. I'm a reliable and hard-working individual who is always ready to help others. I believe in [B], and I'm constantly striving to improve myself and reach new heights. I'm an outgoing and confident person, and I'm always ready to meet new people and create meaningful connections. I'm excited to meet you, and I look forward to our future together. What's your name, and what do you do? [Name

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 
The statement is concise, and it provides the key information about the capital city in a clear and straightforward manner. It uses the definite article "the" and the singular pronoun "the," and it uses the plural form "capital city" to ind

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 Integr

ating

 emotion

-aware

 AI

In [6]:
llm.shutdown()